In [ ]:
#this script creates file 'plfull.pkl' with all the available data about players in ttligapro database (currently 661 players,
#though data about 200 players was deleted from the database), and on all the matches at set level of the players in the database

#import everything I need
import requests
import pickle
import numpy as np
import pandas as pd
import time
from bs4 import BeautifulSoup

In [ ]:
class player: #class for players
    def __init__(self):
        self
class game: #each object - a game between two players
    def __init__(self):
        self

def ttligaproplayers():
    global players
    #Parse the page with players and create a table
    tournpage = requests.get('http://tt.sport-liga.pro/players/?rat1=0&rat2=2700&rat=Search#list')
    tournpage = BeautifulSoup(tournpage.text, 'html.parser')
    rows = tournpage('table')[1].findAll('tr')[1:]
    table = [[cell.text for cell in row('td')] for row in rows]
    tournpagetable = tournpage('table')[1]
    if table == []:
        rows = tournpage('table')[0].findAll('tr')[1:]
        table = [[cell.text for cell in row('td')] for row in rows]
        tournpagetable = tournpage('table')[0]
    players = [] #list of players
    a = [] #list of links to get players' ids
    j = 0
    for p in tournpagetable('a'):
        if j%2==0:
            a.append(p['href'][8:]) #contains id of each player
            print(p['href'][8:])
        j+=1    
    for i in range(len(table)):
        p = player() #create player object
        p.ranking = table[i][0] #add ranking
        p.name = table[i][1] #add name
        p.sex = table[i][2] #add sex
        p.login = table[i][3] #add login
        p.rating = table[i][4] #add rating
        p.date = table[i][5] #add date
        p.fntrrating = table[i][6] #add fntrrating
        p.fntrdate = table[i][7] #add fntrdate
        p.id = a[i] #add id
        players.append(p)  
ttligaproplayers()    

In [ ]:
#returns list of all competitors of the player with id=id
def competitors(id):
    page = requests.get('http://tt-liga.pro/statistics/'+id)
    page = BeautifulSoup(page.text, 'html.parser')
    rows = page('table')[0].findAll('tr')[1:]
    table = [[cell.string for cell in row('td')] 
             for row in rows]
    pagetable = page('table')[0]
    if table==[]:
        rows = page('table')[1].findAll('tr')[1:]
        table = [[cell.string for cell in row('td')] 
                 for row in rows]
        pagetable = page('table')[1]
    a = [] 
    i = 0
    for p in pagetable('a'):
        if i%3==0:
            a.append(p['href'][8:])
        i+=1    
    return(a)  
def getplays(a,b):
    global page
    while True:#since there is a probability of a connection error
        try:
            page = requests.get('http://tt-liga.pro/statistics/'+a+'?vs='+b)
            page = BeautifulSoup(page.text, 'html.parser')
            break
        except requests.ConnectionError:
            print('ce:(')
        except:
            print('someerror')
def createplaystable(player, comp):
    global page
    #print(comp)
    for c in comp:
        getplays(player.id, c)
        #print(c)
        try:
            rows = page('table')[0].findAll('tr')[1:-1]
            table = [[cell.text for cell in row('td')] for row in rows] 
            if table==[]:
                rows = page('table')[1].findAll('tr')[1:-1]
                table = [[cell.text for cell in row('td')] for row in rows] 
            #print(table)
        except: #occurs if there were no games between the players
            table = 'None'
        try:
            player.against.update({c: table})
        except: #occurs when player.against is not created yet
            player.against = {c: table}

#functions for files    
def writefile(file, data):
    with open(file+'.pkl', 'wb') as output:
        #save = data
        pickle.dump(data, output, pickle.HIGHEST_PROTOCOL)
        
def readfile(file):
    with open(file+'.pkl', 'rb') as input:
        return(pickle.load(input))    

In [ ]:
for pl in players: 
    o = competitors(pl.id)
    createplaystable(pl,o)
    print(pl.name)
    writefile('plfull', players)